In [8]:
import pandas as pd

In [9]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [10]:
# Load dataset
def load_data():
    data =pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/tweet_covid_processing_file2.csv", engine='python')
    return data

In [11]:
tweet_df = load_data()
tweet_df.head(5)

,Unnamed: 0,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,1,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,2,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,3,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
tweet_df.drop('Unnamed: 0',inplace=True,axis = 1)

In [14]:
tweet_df.head(5)

,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [15]:
tweet_df= tweet_df.dropna()

In [16]:
X=tweet_df.drop('label',axis=1)

In [17]:
type(X)

pandas.core.frame.DataFrame

In [18]:
## Get the Dependent features
y=tweet_df['label']

In [19]:
X.head()

,OriginalTweet,Sentiment,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [20]:
X_final = X['tweet_stemmed'].copy()

In [21]:
X_final

0                              menyrbi phil gahan chrisitv
1        advic talk neighbour famili exchang phone numb...
2        coronaviru australia woolworth give elderli di...
3        My food stock one empti pleas panic there will...
4        Me readi go supermarket covid outbreak not I p...
                               ...                        
41134    airlin pilot offer stock supermarket shelv NZ ...
41135    respons complaint provid cite covid relat dela...
41136    you know get tough kameronwild ration toilet p...
41137    Is wrong smell hand sanit start turn coronavir...
41138    tartiicat well new use rift S go amazon rn alt...
Name: tweet_stemmed, Length: 41139, dtype: object

In [22]:
y.value_counts()

1    11421
4     9917
0     7697
2     6623
3     5481
Name: label, dtype: int64

In [23]:
X_final.shape, y.shape

((41139,), (41139,))

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [25]:
### Vocabulary size
voc_size=50000

**One hot representation**

In [26]:
messages= X.copy()

In [27]:
messages['tweet_stemmed'][1]

'advic talk neighbour famili exchang phone number creat contact list phone number neighbour school employ chemist GP set onlin shop account poss adequ suppli regular med order'

In [28]:
type(messages['tweet_lemmatized'][1])

str

In [29]:
messages.reset_index(inplace=True)

In [30]:
onehot_repr=[one_hot(words,voc_size)for words in messages['tweet_stemmed']] 
onehot_repr

[[46340, 27746, 29539, 4419],
 [35919,
  26317,
  16101,
  17484,
  6870,
  3109,
  46471,
  28416,
  12736,
  12357,
  3109,
  46471,
  16101,
  28019,
  4381,
  21720,
  758,
  34604,
  37466,
  2986,
  7568,
  34557,
  27863,
  42390,
  14397,
  4776,
  45198],
 [23228,
  7146,
  836,
  41723,
  31377,
  2565,
  35498,
  2986,
  35949,
  8744,
  11805,
  22526],
 [37186,
  24678,
  18811,
  4616,
  18166,
  5758,
  6809,
  36260,
  3032,
  11466,
  44785,
  24678,
  46950,
  42523,
  33389,
  42875,
  42182,
  19558,
  42182,
  11508,
  11805,
  43156,
  11805,
  11805,
  23228,
  48768,
  21843,
  44046],
 [19195,
  41267,
  42106,
  25249,
  11805,
  22526,
  1250,
  32020,
  6204,
  24678,
  18811,
  19255,
  18166,
  24179,
  23228,
  17903,
  47064,
  5758,
  6809,
  25671,
  2148,
  6791,
  43418,
  42274,
  16123,
  48768],
 [4061,
  26945,
  16325,
  2645,
  14159,
  11805,
  17997,
  26222,
  33027,
  5290,
  33965,
  7820,
  1567,
  19439,
  45564,
  1176,
  16612,
  44186

In [31]:
type(onehot_repr[1][1])

int

**Embedding representation**

In [32]:
sent_length=56
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ... 27746 29539  4419]
 [    0     0     0 ... 14397  4776 45198]
 [    0     0     0 ...  8744 11805 22526]
 ...
 [    0     0     0 ... 44483 45769  7599]
 [    0     0     0 ... 23228 11805 23228]
 [    0     0     0 ... 11805 47021  1987]]


In [33]:
type(embedded_docs[1][19])

numpy.int32

In [34]:
convert_embed= embedded_docs.astype(int)

In [35]:
embedded_docs[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0, 46340, 27746,
       29539,  4419], dtype=int32)

**Creating Model**

In [36]:
# ## Creating model for LSTM
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(LSTM(300, Dropout(0.2),recurrent_dropout=0.2))
# model.add(Dense(1,activation='softmax'))

# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())

**## Creating model for BI-LSTM**

In [37]:
## Creating model for BI-LSTM
from tensorflow.keras.layers import Embedding,SpatialDropout1D
embedding_vector_features=196
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(SpatialDropout1D(0.2))
model1.add(Bidirectional(LSTM(350)))
model1.add(Dropout(0.2))
model1.add(Dense(5,activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 56, 196)           9800000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 56, 196)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 700)               1531600   
_________________________________________________________________
dropout (Dropout)            (None, 700)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 3505      
Total params: 11,335,105
Trainable params: 11,335,105
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
len(embedded_docs),y.shape

(41139, (41139,))

In [39]:
import numpy as np
X_final=np.array(convert_embed)
y_final=np.array(y)

In [40]:
X_final.shape,y_final.shape

((41139, 56), (41139,))

In [41]:
type(y_final[1])

numpy.int64

In [42]:
convert_y_final=y_final.astype(int)

In [43]:
convert_y_final

array([0, 1, 1, ..., 1, 0, 4])

In [44]:
type(convert_y_final[1])

numpy.int64

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, convert_y_final, test_size=0.33, random_state=42)

**Model training**

In [46]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=32)

Epoch 1/5
862/862 [==============================] - 763s 886ms/step - loss: 1.1409 - accuracy: 0.5211 - val_loss: 0.8907 - val_accuracy: 0.6564
Epoch 2/5
862/862 [==============================] - 759s 880ms/step - loss: 0.7476 - accuracy: 0.7207 - val_loss: 0.8575 - val_accuracy: 0.6810
Epoch 3/5
862/862 [==============================] - 750s 870ms/step - loss: 0.5556 - accuracy: 0.7973 - val_loss: 0.9359 - val_accuracy: 0.6670
Epoch 4/5
862/862 [==============================] - 734s 852ms/step - loss: 0.3873 - accuracy: 0.8647 - val_loss: 1.0404 - val_accuracy: 0.6456
Epoch 5/5
862/862 [==============================] - 743s 862ms/step - loss: 0.2777 - accuracy: 0.9064 - val_loss: 1.2072 - val_accuracy: 0.6339


In [47]:
y_pred=model1.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


0.6339127872716559